# Задача на 4-ом шаге урока

**🛻 Как там твоя Tesla?**

In [21]:
!pip install featuretools -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.9/587.9 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.2/215.2 kB 12.8 MB/s eta 0:00:00


In [30]:
import pandas as pd
import featuretools as ft

In [31]:
path = 'https://raw.githubusercontent.com/a-milenkin/Competitive_Data_Science/main/data/car_info.csv'
car_info = pd.read_csv(path)
car_info.head(3)

,car_type,fuel_type,car_rating,year_to_start,riders,car_id,model,target_class,year_to_work,target_reg
0,economy,petrol,4.80,2013,42269,P17494612l,Skoda Rapid,engine_overheat,2019,46.650719
1,standart,petrol,4.32,2015,90014,N-1530212S,Renault Sandero,electro_bug,2016,34.377312
2,economy,petrol,4.46,2015,82684,B-1154399t,Smart ForTwo,break_bug,2017,39.157169


In [44]:
mask = car_info['model'].str.contains('Tesla', case = False)
tesla_ids = car_info[mask]['car_id'].to_list()
tesla_ids

['O-2124190y',
 'n19871201N',
 'I13914989Z',
 'g-2283000q',
 'i-1106718P',
 'I-2073229s',
 'N44849090k',
 'T20813001U',
 'O-2688441F',
 'p22133368b',
 'T19756755G',
 'v22568580b',
 'l-1177682G',
 'h-1921200Q']

In [32]:
data_root = "https://raw.githubusercontent.com/a-milenkin/Competitive_Data_Science/main/data/"

path = data_root + 'car_train.csv'
car_info = pd.read_csv(path)

path = data_root + 'rides_info.csv'
rides_info = pd.read_csv(path)

path = data_root + 'driver_info.csv'
driver_info = pd.read_csv(path)

path = data_root + 'fix_info.csv'
fix_info = pd.read_csv(path)

In [33]:
es = ft.EntitySet(id="car_data")

In [34]:
from woodwork.logical_types import Categorical, Double, Datetime, Age

es = es.add_dataframe(
    dataframe_name="cars",
    dataframe=car_info,
    index="car_id",
    logical_types={"car_type": Categorical, 'fuel_type': Categorical, 'model': Categorical}
    )

es = es.add_dataframe(
    dataframe_name="rides",
    dataframe=rides_info.drop(['ride_id'], axis=1),
    index='index',
    time_index="ride_date",
    )

es = es.add_dataframe(
    dataframe_name="drivers",
    dataframe=driver_info,
    index="user_id",
    logical_types={"sex": Categorical, "first_ride_date": Datetime, "age": Age}
    )

es = es.add_dataframe(
    dataframe_name="fixes",
    dataframe=fix_info,
    index="index",
    logical_types={"work_type": Categorical, "worker_id":Categorical}
    )
es

/usr/local/lib/python3.12/dist-packages/woodwork/type_sys/utils.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
/usr/local/lib/python3.12/dist-packages/woodwork/type_sys/utils.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
/usr/local/lib/python3.12/dist-packages/woodwork/type_sys/utils.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
/usr/local/lib/python3.12/dist-packages/woodwork/type_sys/utils.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. 

Entityset: car_data
  DataFrames:
    cars [Rows: 2337, Columns: 10]
    rides [Rows: 739500, Columns: 14]
    drivers [Rows: 15153, Columns: 7]
    fixes [Rows: 146000, Columns: 7]
  Relationships:
    No relationships

In [35]:
es = es.add_relationship("cars", "car_id", "rides", "car_id")
es = es.add_relationship("drivers", "user_id", "rides", "user_id")
es = es.add_relationship("cars", "car_id", "fixes", "car_id")

es

/usr/local/lib/python3.12/dist-packages/featuretools/entityset/entityset.py:393: UserWarning: Logical type Categorical for child column car_id does not match parent column car_id logical type Unknown. Changing child logical type to match parent.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/featuretools/entityset/entityset.py:393: UserWarning: Logical type Categorical for child column user_id does not match parent column user_id logical type Unknown. Changing child logical type to match parent.
  warnings.warn(


Entityset: car_data
  DataFrames:
    cars [Rows: 2337, Columns: 10]
    rides [Rows: 739500, Columns: 14]
    drivers [Rows: 15153, Columns: 7]
    fixes [Rows: 146000, Columns: 7]
  Relationships:
    rides.car_id -> cars.car_id
    rides.user_id -> drivers.user_id
    fixes.car_id -> cars.car_id

In [45]:
feature_matrix, feature_defs = ft.dfs(
    entityset=es,
    target_dataframe_name="cars",
    max_depth=1,
    agg_primitives = ["sum", "count", "max"],
    instance_ids = tesla_ids
)
feature_matrix.head()

/usr/local/lib/python3.12/dist-packages/featuretools/computational_backends/feature_set_calculator.py:785: FutureWarning: The provided callable <function sum at 0x7aceac74f880> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  ).agg(to_agg)
/usr/local/lib/python3.12/dist-packages/featuretools/computational_backends/feature_set_calculator.py:785: FutureWarning: The provided callable <function max at 0x7aceac74ff60> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  ).agg(to_agg)
/usr/local/lib/python3.12/dist-packages/featuretools/computational_backends/feature_set_calculator.py:785: FutureWarning: The provided callable <function sum at 0x7aceac74f880> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will b

,model,car_type,fuel_type,car_rating,year_to_start,riders,year_to_work,target_reg,target_class,COUNT(rides),...,SUM(rides.ride_duration),SUM(rides.speed_avg),SUM(rides.speed_max),SUM(rides.stop_times),SUM(rides.user_ride_quality),COUNT(fixes),MAX(fixes.destroy_degree),MAX(fixes.work_duration),SUM(fixes.destroy_degree),SUM(fixes.work_duration)
car_id,,,,,,,,,,,,,,,,,,,,,
O-2124190y,Tesla Model 3,premium,electro,3.90,2017,116872,2019,50.26,gear_stick,174,...,215095.0,7630.0,11601.438521,147.0,-707.814812,35,10.0,61.0,113.1,970.0
n19871201N,Tesla Model 3,premium,electro,3.58,2016,89811,2018,100.42,another_bug,174,...,236549.0,7786.0,16070.296888,694.0,1127.610285,35,10.0,68.0,124.6,858.0
I13914989Z,Tesla Model 3,premium,electro,4.64,2016,98304,2019,100.62,engine_ignition,174,...,290518.0,8935.0,16999.476225,139.0,1929.941778,35,5.0,65.0,115.0,896.0
g-2283000q,Tesla Model 3,premium,electro,5.44,2015,75113,2020,47.95,gear_stick,174,...,319610.0,7845.0,11860.793108,131.0,-380.108901,35,9.3,60.0,119.0,796.0
i-1106718P,Tesla Model 3,premium,electro,4.72,2015,75786,2017,119.84,engine_ignition,174,...,330058.0,8978.0,16625.210818,134.0,-2640.911012,35,5.0,65.0,92.4,914.0


In [50]:
feature_matrix.columns

Index(['model', 'car_type', 'fuel_type', 'car_rating', 'year_to_start',
       'riders', 'year_to_work', 'target_reg', 'target_class', 'COUNT(rides)',
       'MAX(rides.deviation_normal)', 'MAX(rides.distance)',
       'MAX(rides.rating)', 'MAX(rides.refueling)', 'MAX(rides.ride_cost)',
       'MAX(rides.ride_duration)', 'MAX(rides.speed_avg)',
       'MAX(rides.speed_max)', 'MAX(rides.stop_times)',
       'MAX(rides.user_ride_quality)', 'SUM(rides.deviation_normal)',
       'SUM(rides.distance)', 'SUM(rides.rating)', 'SUM(rides.refueling)',
       'SUM(rides.ride_cost)', 'SUM(rides.ride_duration)',
       'SUM(rides.speed_avg)', 'SUM(rides.speed_max)', 'SUM(rides.stop_times)',
       'SUM(rides.user_ride_quality)', 'COUNT(fixes)',
       'MAX(fixes.destroy_degree)', 'MAX(fixes.work_duration)',
       'SUM(fixes.destroy_degree)', 'SUM(fixes.work_duration)'],
      dtype='object')

In [60]:
len(feature_matrix.index)

14

In [53]:
feature_matrix['MAX(rides.speed_max)'].max()

199.3701025284091

In [64]:
round(feature_matrix['COUNT(fixes)'].mean(), 1)

np.float64(34.4)

In [57]:
feature_matrix['SUM(fixes.work_duration)'].max()

970.0

In [63]:
!pip install skimpy -q
from skimpy import clean_columns


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.5/117.5 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 43.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires ipykernel==6.17.1, but you have ipykernel 6.30.1 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.2 which is incompatible.
cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.2 which is incompatible.
notebook 6.5.7 requires jupyter-client<8,>=5.3.4, but you have jupyter-client 8.6.3 which is incompatible.
jupyter-kernel-gateway 2.5.2 re

# Задача на 7-ом шаге урока

**🤩 Выбираем себе дом в LA 🌴**

In [ ]:
# dataset california housing

In [ ]:
!pip install geopandas -q
import geopandas as gpd
from sklearn.datasets import fetch_california_housing

In [ ]:
df = fetch_california_housing(as_frame=True).data
df.head(3)

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24


In [ ]:
# создадим GeoDataFrame
gdf = gpd.GeoDataFrame(
        df,
        geometry=gpd.points_from_xy(df['Longitude'], df['Latitude']),
        crs=4326
    ).to_crs(epsg=3857)
gdf.head(3)

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,geometry
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,POINT (-13606581.360 4562487.679)
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,POINT (-13605468.165 4559667.342)
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,POINT (-13607694.555 4558257.461)


In [ ]:
import requests, zipfile, io

county_fname = 'https://github.com/a-milenkin/Competitive_Data_Science/raw/main/data/ca-county-boundaries.zip'
r2 = requests.get(county_fname)
z2 = zipfile.ZipFile(io.BytesIO(r2.content))
z2.extractall("./ca")
ca_counties=gpd.read_file('./ca/CA_Counties').to_crs("EPSG:3857")
gdf = gpd.overlay(gdf, ca_counties[['NAME', 'geometry']], how='intersection')
gdf.head(3)


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,NAME,geometry
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,Alameda,POINT (-13606581.360 4562487.679)
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,Alameda,POINT (-13605468.165 4559667.342)
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,Alameda,POINT (-13607694.555 4558257.461)


In [ ]:
# Максимальное расстояние до центра LA

# Минимальное расстояние до центра LA

# Среднее расстояние до центра LA

# Задача на 8-ом шаге урока

**🤯 Не курс, а Санта-Барбара какая-то...**

Для отладки воспользуйтесь примером и датасетом (ca_counties) из демонстрационного ноутбука (или из прошлой задачи):

In [ ]:
ca_counties.head(3)

,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,06,091,00277310,06091,Sierra,Sierra County,06,H1,G4020,NaN,NaN,NaN,A,2468694587,23299110,+39.5769252,-120.5219926,"POLYGON ((-13431319.751 4821511.426, -13431312..."
1,06,067,00277298,06067,Sacramento,Sacramento County,06,H1,G4020,472,40900,NaN,A,2499183617,76073827,+38.4500114,-121.3404409,"POLYGON ((-13490651.476 4680831.603, -13490511..."
2,06,083,00277306,06083,Santa Barbara,Santa Barbara County,06,H1,G4020,NaN,42200,NaN,A,7084000598,2729814515,+34.5370572,-120.0399729,"MULTIPOLYGON (((-13423116.772 4042044.149, -13..."


In [ ]:
# Заполните словарь params верными параметрами и их значениями.
# Датасет ca_counties уже записан в переменную ca_counties.

params_1 = {'geo_data': ca_counties,
            '?': ca_counties,
            'columns': ['?', '?'],
            '?': 'feature.properties.NAME',
            '?': '?',
            '?': 'YlGn'}

params_2 = {'?':[34.4208, -119.698],
            '?':5,
            'color':'?',
            '?':'?'}

In [ ]:
import folium

m = folium.Map([37, -122], tiles='OpenStreetMap', zoom_start=6)

# наносим на карту округа с заливкой по площади водного пространства
folium.Choropleth(**params_1).add_to(m)

# наносим на карту город Санта-Барбара
folium.CircleMarker(**params_2).add_to(m)

m

# Задача на 13-ом шаге урока

**Модифицируйте MinimalFCParameters из TSFresh**

In [1]:
!pip install tsfresh -q

In [2]:
# Посмотрим на набор признаков в MinimalFCParameters
from tsfresh.feature_extraction import MinimalFCParameters

In [3]:
fc_parameters = {
    'maximum' : None,
    'minimum' : None,
    'standard_deviation' : None,
    'variance' : None,
    'linear_trend': [{'attr': 'pvalue'}, {'attr': 'rvalue'}, {'attr': 'intercept'}, {'attr': 'slope'}],
    'quantile' : [{'q': 0.25}, {'q': 0.75}],
    'number_peaks' : [{'n' : 2}]
}

In [4]:
from tsfresh.examples.robot_execution_failures import download_robot_execution_failures, \
    load_robot_execution_failures

In [5]:
download_robot_execution_failures()
timeseries, y = load_robot_execution_failures()
timeseries

,id,time,F_x,F_y,F_z,T_x,T_y,T_z
0,1,0,-1,-1,63,-3,-1,0
1,1,1,0,0,62,-3,-1,0
2,1,2,-1,-1,61,-3,0,0
3,1,3,-1,-1,63,-2,-1,0
4,1,4,-1,-1,63,-3,-1,0
...,...,...,...,...,...,...,...,...
1315,88,10,-10,2,39,-21,-24,5
1316,88,11,-11,2,38,-24,-22,6
1317,88,12,-12,3,23,-24,-24,5
1318,88,13,-13,4,26,-29,-27,5


In [8]:
from pprint import pprint

pprint(MinimalFCParameters())

{'absolute_maximum': None,
 'length': None,
 'maximum': None,
 'mean': None,
 'median': None,
 'minimum': None,
 'root_mean_square': None,
 'standard_deviation': None,
 'sum_values': None,
 'variance': None}


In [9]:
from tsfresh import extract_features
extracted_features = extract_features(timeseries, column_id="id", column_sort="time", n_jobs=2, default_fc_parameters=MinimalFCParameters())
extracted_features.head()

Feature Extraction: 100%|██████████| 10/10 [00:00<00:00, 23.22it/s]


,F_x__sum_values,F_x__median,F_x__mean,F_x__length,F_x__standard_deviation,F_x__variance,F_x__root_mean_square,F_x__maximum,F_x__absolute_maximum,F_x__minimum,...,T_z__sum_values,T_z__median,T_z__mean,T_z__length,T_z__standard_deviation,T_z__variance,T_z__root_mean_square,T_z__maximum,T_z__absolute_maximum,T_z__minimum
1,-14.0,-1.0,-0.933333,15.0,0.249444,0.062222,0.966092,0.0,1.0,-1.0,...,0.0,0.0,0.000000,15.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
2,-13.0,-1.0,-0.866667,15.0,0.956847,0.915556,1.290994,0.0,3.0,-3.0,...,-4.0,0.0,-0.266667,15.0,0.442217,0.195556,0.516398,0.0,1.0,-1.0
3,-10.0,-1.0,-0.666667,15.0,0.596285,0.355556,0.894427,1.0,1.0,-1.0,...,-4.0,0.0,-0.266667,15.0,0.442217,0.195556,0.516398,0.0,1.0,-1.0
4,-6.0,0.0,-0.400000,15.0,0.952190,0.906667,1.032796,1.0,2.0,-2.0,...,-5.0,0.0,-0.333333,15.0,0.596285,0.355556,0.683130,1.0,1.0,-1.0
5,-9.0,-1.0,-0.600000,15.0,0.879394,0.773333,1.064581,2.0,2.0,-2.0,...,-2.0,0.0,-0.133333,15.0,0.618241,0.382222,0.632456,1.0,1.0,-1.0


In [19]:
fc_parameters = MinimalFCParameters()
for f in ["minimum", "maximum", "standard_deviation", "variance"]:
    fc_parameters.pop(f, None)
fc_parameters.update(
     {
    'linear_trend': [{'attr': 'pvalue'}, {'attr': 'rvalue'}, {'attr': 'intercept'}, {'attr': 'slope'}],
    'quantile' : [{'q': 0.25}, {'q': 0.75}],
    'number_peaks' : [{'n' : 2}]
}
)
extracted_features = extract_features(timeseries, column_id="id", column_sort="time", n_jobs=2, default_fc_parameters= fc_parameters)
extracted_features.head()

Feature Extraction: 100%|██████████| 10/10 [00:00<00:00, 15.72it/s]


,F_x__sum_values,F_x__median,F_x__mean,F_x__length,F_x__root_mean_square,F_x__absolute_maximum,"F_x__linear_trend__attr_""pvalue""","F_x__linear_trend__attr_""rvalue""","F_x__linear_trend__attr_""intercept""","F_x__linear_trend__attr_""slope""",...,T_z__length,T_z__root_mean_square,T_z__absolute_maximum,"T_z__linear_trend__attr_""pvalue""","T_z__linear_trend__attr_""rvalue""","T_z__linear_trend__attr_""intercept""","T_z__linear_trend__attr_""slope""",T_z__quantile__q_0.25,T_z__quantile__q_0.75,T_z__number_peaks__n_2
1,-14.0,-1.0,-0.933333,15.0,0.966092,1.0,0.173193,-3.711537e-01,-0.783333,-2.142857e-02,...,15.0,0.000000,0.0,NaN,NaN,0.000000,0.000000e+00,0.0,0.0,0.0
2,-13.0,-1.0,-0.866667,15.0,1.290994,3.0,1.000000,-7.161487e-18,-0.866667,-1.586033e-18,...,15.0,0.516398,1.0,1.000000,1.355872e-17,-0.266667,1.387779e-18,-0.5,0.0,0.0
3,-10.0,-1.0,-0.666667,15.0,0.894427,1.0,0.713543,-1.035098e-01,-0.566667,-1.428571e-02,...,15.0,0.516398,1.0,0.619819,-1.395726e-01,-0.166667,-1.428571e-02,-0.5,0.0,0.0
4,-6.0,0.0,-0.400000,15.0,1.032796,2.0,0.774068,-8.102547e-02,-0.275000,-1.785714e-02,...,15.0,0.683130,1.0,1.000000,-1.867434e-17,-0.333333,-2.577303e-18,-1.0,0.0,1.0
5,-9.0,-1.0,-0.600000,15.0,1.064581,2.0,0.755869,-8.773280e-02,-0.475000,-1.785714e-02,...,15.0,0.632456,1.0,0.201747,-3.494181e-01,0.216667,-5.000000e-02,-0.5,0.0,0.0


In [20]:
pprint(fc_parameters)

{'absolute_maximum': None,
 'length': None,
 'linear_trend': [{'attr': 'pvalue'},
                  {'attr': 'rvalue'},
                  {'attr': 'intercept'},
                  {'attr': 'slope'}],
 'mean': None,
 'median': None,
 'number_peaks': [{'n': 2}],
 'quantile': [{'q': 0.25}, {'q': 0.75}],
 'root_mean_square': None,
 'sum_values': None}
